<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 19 : Requêtes SQL et mises à jour</h1>

Nous avons vu comment créer des tables et les remplir.  
Nous avons maintenant une base de données, c'est-à-dire un ensemble de tables, contenant des données **cohérentes** vis à vis de nos contraintes d'intégrité.  

Nous allons maintenant voir deux autres utilisations d'un SGBD :
* la sélection de données
* la mise à jour des données

La sélection va consister en l'écriture de **requêtes SQL** permettant de trouver toutes les données de la base vérifiant un certain critère.  
Le premier rôle du programmeur de bases de données va donc être celui qui consiste à traduire des questions que l'on se pose sur les données du langage naturel au langage SQL, afin que le SGBD puisse y répondre.  

Reprenons le fil conducteur de notre introduction aux bases de données, à savoir celui de la médiathèque municipale.  
Quelles questions peut-on poser à la base de données?  
Il semble naturel que le SGBD puisse répondre aux questions suivantes, nécessaire au bon fonctionnement
de la médiathèque.
* Étant donné un code barre, quels sont les livres empruntés par l'utilisateur correspondant?
* Étant donné un ISBN, le livre correspondant est-il emprunté?
* Quels sont les utilisateurs en retard, c'est-à-dire ceux dont la date de retour est inférieure à une date donnée?
* Quels sont tous les livres écrits par Voltaire qui ne sont pas empruntés?
* Quel est le nombre total de livres empruntés?

Une autre fonction importante du SGBD est la mise à jour des données.  
Elle peut consister en une modification d'une ligne existante (par exemple, pour changer l'adresse d'un utilisateur ayant déménagé, sans modifier son code barre, son nom ou son e-mail) ou une suppression (par exemple lorsqu'un utilisateur rend un livre, il faut supprimer la ligne correspondante dans la table `emprunt`).

## Sélection de données

### Requête sur une table
Commençons par une requête simple. On considère la table [`livre`](Fichiers/livre.csv), créée par l'ordre suivant:

```sql
CREATE TABLE livre (titre VARCHAR(300) NOT NULL, editeur VARCHAR(90) NOT NULL, 
                     annee INT NOT NULL, isbn CHAR(14) PRIMARY KEY);
```

On souhaite trouver [les titres de tous les livres publiés après 1990 dans la base de données de la médiathèque](Fichiers/resultat_1.csv).  
Une telle requête peut s'écrire en SQL :

```sql
SELECT titre FROM livre 
WHERE annee >= 1990;
```

|               titre               |
|:---------------------------------:|
| Les Aventures de Huckleberry Finn |
|        Fondation et Empire        |
|               Akira               |
|             Les Robots            |
|      Astérix chez les Pictes      |
|        Les Monades urbaines       |
| ...                               |

```

Result: 112 enregistrements ramenés en 9ms
```

Dans cette requête, la partie `FROM livre` indique la table sur laquelle porte la requête.  
La partie `WHERE ...` indique que l'on ne sélectionne que les lignes de la table `livre` pour lesquelles la valeur de l'attribut `annee` est plus grande que 1990.  
Enfin, la partie `SELECT titre` indique qu'on ne veut renvoyer que les valeurs de l'attribut `titre` des lignes trouvées.  
On remarque que le résultat d'une requête `SELECT ...` est une table, ici possédant une unique colonne `titre`.

### Clause WHERE
L'expression se trouvant dans la partie `WHERE` doit être une expression booléenne.  
Elle peut être construite à partir d'opérateurs de comparaison (`<`, `<=`, `>`, `>=`, `=` et `<>`), d'opérateurs arithmétiques (`+`, `-`, `*`, `/`, `%`), de constantes, de noms d'attributs, d'opérateurs logiques (`AND`, `OR` et `NOT`) et d'opérateurs spéciaux tels que l'opérateur de comparaison de textes `LIKE`.
Par exemple, si l'on souhaite afficher [les titres de tous les livres publiés par Dargaud entre 1970 et 1980](Fichiers/resultat_2.csv), on pourra écrire:

```sql
SELECT titre FROM livre 
WHERE annee >= 1970 AND annee <= 1980 AND editeur = 'Dargaud';
```

|               titre               |
|:---------------------------------:|
| Astérix chez les Belges |

```

Result: 1 enregistrements ramenés en 3ms
```

Si l'utilisation de l'égalité `=` est appropriée ici, on pourrait vouloir faire une requête approchée.  
Par exemple, trouver [les titres des livres qui contiennent le mot `'Astérix'` dans le titre](Fichiers/resultat_3.csv).  
Une telle requête s'écrira :

```sql
SELECT titre FROM livre 
WHERE titre LIKE '%Astérix%';
```

|            titre            |
|:---------------------------:|
|   Astérix chez les Pictes   |
|     Astérix et Cléopâtre    |
|  Le Tour de Gaule d'Astérix |
|   Astérix et les Normands   |
|       Astérix en Corse      |
|     Astérix légionnaire     |
| Astérix et la Transitalique |
|     L'Odyssée d'Astérix     |
|   Astérix chez les Bretons  |
|   Astérix chez les Belges   |

```
Result: 10 enregistrements ramenés en 4ms
```



### Clause SELECT
La clause `SELECT` peut prendre diverses formes.
* La première est celle où l'on liste explicitement les attributs que l'on désire renvoyer.  
Si on veut renvoyer [les titres et l'ISBN des livres publiés après 1990](Fichiers/resultat_4.csv), on écrira ceci:

```sql
SELECT titre, isbn FROM livre 
WHERE annee >= 1990;
```

|               titre               |      isbn      |
|:---------------------------------:|:--------------:|
| Les Aventures de Huckleberry Finn | 978-2081509511 |
| Fondation et Empire               | 978-2207249123 |
| Akira                             | 978-2723428262 |
| Les Robots                        | 978-2745989857 |
| Astérix chez les Pictes           | 978-2864972662 |
| Les Monades urbaines              | 978-2221197691 |
| Les Voyages de Gulliver           | 978-2335008586 |
| Lolita                            | 978-0141391601 |
| La Nuit des temps                 | 978-2258116429 |
| Ravage                            | 978-2072534911 |
| Les Lauriers de César             | 978-2012101500 |
|                ...                |                |

```
Result: 112 enregistrements ramenés en 17ms
```

Le résultat est de nouveau une table, mais cette fois avec les colonnes (ou attributs) `titre` et `isbn`.  
Il est possible de renommer les colonnes au moyen du mot clé `AS`.

```sql
SELECT titre AS le_titre, isbn AS num_serie FROM livre 
WHERE annee >= 1990;
```


|               le_titre               |      num_serie      |
|:---------------------------------:|:--------------:|
| Les Aventures de Huckleberry Finn | 978-2081509511 |
| Fondation et Empire               | 978-2207249123 |
| Akira                             | 978-2723428262 |
| Les Robots                        | 978-2745989857 |
| Astérix chez les Pictes           | 978-2864972662 |
| Les Monades urbaines              | 978-2221197691 |
| Les Voyages de Gulliver           | 978-2335008586 |
| Lolita                            | 978-0141391601 |
| La Nuit des temps                 | 978-2258116429 |
| Ravage                            | 978-2072534911 |
| Les Lauriers de César             | 978-2012101500 |
|                ...                |                |

```
Result: 112 enregistrements ramenés en 13ms
```

Ainsi, si la clause `WHERE` d'une requête permet de restreindre les lignes de la table que l'on renvoie (en ne gardant que celle vérifiant la condition), la clause `SELECT` permet de restreindre la liste des colonnes.

* La seconde forme de la clause `SELECT` est celle que l'on utilise lorsqu'on veut conserver [toutes les colonnes](Fichiers/resultat_5.csv). En effet, il serait fastidieux de récrire toutes les colonnes d'une table.  
On peut utiliser à cette fin le symbole `*` qui signifie `toutes les colonnes de la table`.

```sql
SELECT * FROM livre 
WHERE annee >= 1990;
```

| titre                             | editeur              | annee | isbn           |
|:-----------------------------------:|:----------------------:|:-------:|:----------------:|
| Les Aventures de Huckleberry Finn | Flammarion           | 2020  | 978-2081509511 |
| Fondation et Empire               | Editions Denoël      | 1999  | 978-2207249123 |
| Akira                             | Glénat               | 2000  | 978-2723428262 |
| Les Robots                        | Editions Milan       | 2017  | 978-2745989857 |
| Astérix chez les Pictes           | Editions Albert René | 2013  | 978-2864972662 |
| Les Monades urbaines              | Robert Laffont       | 2016  | 978-2221197691 |
| Les Voyages de Gulliver           | Primento             | 2015  | 978-2335008586 |
| Lolita                            | Penguin UK           | 2012  | 978-0141391601 |
| La Nuit des temps                 | Presses de la Cité   | 2014  | 978-2258116429 |
| Ravage                            | Editions Gallimard   | 2014  | 978-2072534911 |
| Les Lauriers de César             | Educa Books          | 2008  | 978-2012101500 |
| ...                               |                      |       |                |

```
Result: 112 enregistrements ramenés en 18ms
```

* La troisième utilisation de la clause `SELECT` est celle permettant d'appeler des **fonctions d'agrégation**.  
Ces dernières permettent d'appliquer une fonction à l'ensemble des valeurs d'une colonne et de renvoyer le résultat comme une table ayant une seule case (une ligne et une colonne).   
Voici quelques unes de ces fonctions: 
    * `COUNT` qui permet d'obtenir le nombre de résultats
    * `AVG` (pour l'anglais average) qui permet de calculer la moyenne d'une colonne
    * `SUM` qui permet d'en faire la somme
    * `MIN` et `MAX` qui permettent de trouver respectivement le minimum et maximum d'une colonne. 
    
Si on souhaite savoir [combien de livres contiennent la chaîne `'Astérix'` dans leur titre](Fichiers/resultat_6.csv) (plutôt que de renvoyer ces titres), on écrira la requête suivante:

```sql
SELECT COUNT(titre) AS total FROM livre 
WHERE titre LIKE '%Astérix%';
```

| total                             | 
|:-----------------------------------:|
| 10 |

```
Result: 1 enregistrements ramenés en 2ms
```

Notons que nous avons choisi de renommer la colonne.  
En effet, le résultat n'étant pas directement une colonne d'une table existante, les SGBD choisissent un nom arbitraire, souvent peu parlant.  
La fonction `COUNT` ne faisant que compter la taille de la colonne, elle peut s'appliquer à n'importe quel nom de colonne et même au symbole `*`.  
Il est donc courant d'écrire une requête de la forme

```sql
SELECT COUNT(*) AS total FROM livre 
WHERE titre LIKE '%Astérix%';
```
qui donnera le même résultat que précédemment.  
Les fonctions `AVG` et `SUM` ne peuvent s'appliquer qu'à des colonnes dont le domaine est un nombre.  
On peut écrire par exemple

```sql
SELECT SUM(annee) AS somme FROM livre;
```

| somme                             | 
|:-----------------------------------:|
| 256701 |

```
Result: 1 enregistrements ramenés en 6ms
```

```sql
SELECT AVG(annee) AS moyenne FROM livre;
```

| moyenne                             | 
|:-----------------------------------:|
| 2005.4765625 |

```
Result: 1 enregistrements ramenés en 7ms
```

Ici, en l'absence de clause `WHERE`, toutes les lignes sont sélectionnées.  
La somme et la moyenne des années sont calculées et renvoyées comme une table.  

Enfin, les fonctions `MIN()` et `MAX()` peuvent s'appliquer sur n'importe quelle colonne et la comparaison pour son type sera utilisée pour déterminer le plus petit ou le plus grand élément.

```sql
SELECT MIN(annee) AS inf FROM livre;
```

| inf                             | 
|:-----------------------------------:|
| 1933 |

```
Result: 1 enregistrements ramenés en 2ms
```

```sql
SELECT MAX(annee) AS sup FROM livre;
```

| sup                             | 
|:-----------------------------------:|
| 2020 |

```
Result: 1 enregistrements ramenés en 2ms
```




### Tri et suppression des doublons
Comme nous avons pu l'observer lors de nos premières requêtes, les résultats sont affichés par le SGBD dans un
ordre a priori quelconque.  
La situation est même plus complexe. En effet, en fonction de certains paramètres, le SGBD peut choisir entre différentes façons de calculer la requête.  
L'ordre peut donc être modifié entre deux exécutions de la même requête.  
Si l'on désire obtenir les résultats dans un ordre particulier, on peut utiliser la clause `ORDER BY` en fin de [requête](Fichiers/resultat_7.csv).

```sql
SELECT titre FROM livre 
WHERE annee >= 1990 
ORDER BY titre ASC;
```
|            titre            |
|:---------------------------:|
| Akira                       |
| Algorithms                  |
| Anna Karénine               |
| Astérix chez les Bretons    |
| Astérix chez les Pictes     |
| Astérix en Corse            |
| Astérix et Cléopâtre        |
| Astérix et la Transitalique |
| Astérix et les Normands     |
| Astérix légionnaire         |
| Au carrefour des étoiles    |
| ... |

```
Result: 112 enregistrements ramenés en 12ms
```

Ici, on demande au SGBD de trier les résultats par titre croissants (`ASC` pour l'anglais *ascending*). Si on souhaite trier par valeurs décroissantes il suffit d'utiliser le mot clé `DESC` (pour l'anglais *descending*) à la place de `ASC`.
Supposons maintenant que l'on souhaite connaître [toutes les années dans lesquelles un livre a été publié](Fichiers/resultat_8.csv).  
La requête :

```sql
SELECT annee FROM livre;
```

nous donne toutes les années, mais la même année peut apparaître plusieurs fois.  
Si on souhaite retirer les doublons d'un résultat, le mot clé `DISTINCT` peut être ajouté à la clause `SELECT`, comme pour la [requête](Fichiers/resultat_9.csv) suivante :

```sql
SELECT DISTINCT annee FROM livre;
```

Attention cependant, chaque ligne entière de résultat est considérée lors de la comparaison.  
C'est pourquoi, la [requête](Fichiers/resultat_10.csv) 

```sql
SELECT DISTINCT annee, isbn FROM livre;
```
continuera d'afficher plusieurs fois la même année.  
En effet, comme l'`isbn` est unique pour chaque ligne, tous les couples `annee`, `isbn` de la table sont
différents deux à deux (ils diffèrent par leur isbn même s'ils ont la même année).  
Le mot clé `DISTINCT` n'aura donc ici aucun effet.

### Jointure
Les requêtes que nous avons vues nous permettent assez facilement de déterminer les livres qui ont été empruntés.  
Ces derniers sont simplement ceux dont l'ISBN est présent dans la table [`emprunt`](Fichiers/emprunt.csv).

```sql
SELECT * FROM emprunt;
```

|    code_barre   |      isbn      |   retour   |
|:---------------:|:--------------:|:----------:|
| 421921003090881 | 978-2081358881 | 2020-04-28 |
| 421921003090881 | 978-2207249123 | 2020-04-28 |
| 421921003090881 | 978-2824709420 | 2020-04-28 |
| 137332830764072 | 978-2352879183 | 2020-02-20 |
| 137332830764072 | 978-2335008586 | 2020-02-20 |
| 137332830764072 | 978-2013230827 | 2020-02-20 |
| 533299198788609 | 978-2253174561 | 2020-02-28 |
| 533299198788609 | 978-2251013039 | 2020-02-28 |
| 917547585216771 | 978-2290105504 | 2020-04-07 |
| 654834075188732 | 978-2864973270 | 2020-02-17 |
| 654834075188732 | 978-2070406340 | 2020-02-17 |
| 654834075188732 | 978-2806231697 | 2020-02-17 |
| 934701281931582 | 978-2260019183 | 2020-01-01 |
| 934701281931582 | 978-2371240087 | 2020-01-01 |
| 035184062854281 | 978-2745989857 | 2020-02-18 |
| 035184062854281 | 978-2072762093 | 2020-02-18 |
| 035184062854281 | 978-2742744824 | 2020-02-18 |

```
Result: 17 enregistrements ramenés en 7ms
```

Cette réponse n'est cependant pas très satisfaisante. En effet, il serait plus naturel de pouvoir afficher les titres de ces livres plutôt que leur ISBN.  
Le problème est que les titres des livres sont présents uniquement dans la table `livre`. L'opération de **jointure** de deux tables apporte une réponse à ce problème.  
Elle a déjà été étudiée en première dans le cadre du traitement de données en tables. Étant données deux tables A et B, la **jointure** consiste à créer toutes combinaisons de lignes de A et de B ayant un attribut de même valeur.  
Ici, on souhaiterait obtenir une *grande table* dont les colonnes sont celles de la table `emprunt` et celle de la table `livre`, en réunissant les lignes ayant le même `isbn`.  
Cela peut être fait au moyen de la directive `JOIN`.

```sql
SELECT * FROM emprunt JOIN livre ON emprunt.isbn = livre.isbn;
```

|    code_barre   |      isbn      |   retour   |            titre            |           editeur           | annee |      isbn      |
|:---------------:|:--------------:|:----------:|:---------------------------:|:---------------------------:|:-----:|:--------------:|
| 421921003090881 | 978-2081358881 | 2020-04-28 |         Mrs Dalloway        |          Flammarion         |  2015 | 978-2081358881 |
| 421921003090881 | 978-2207249123 | 2020-04-28 |     Fondation et Empire     |       Editions Denoël       |  1999 | 978-2207249123 |
| 421921003090881 | 978-2824709420 | 2020-04-28 |     Le Journal d'un fou     |           Bibebook          |  2013 | 978-2824709420 |
| 137332830764072 | 978-2352879183 | 2020-02-20 |        Guerre et Paix       |          Archipoche         |  2016 | 978-2352879183 |
| 137332830764072 | 978-2335008586 | 2020-02-20 |   Les Voyages de Gulliver   |           Primento          |  2015 | 978-2335008586 |
| 137332830764072 | 978-2013230827 | 2020-02-20 |   Gargantua et Pantagruel   |   Livre de Poche Jeunesse   |  2009 | 978-2013230827 |
| 533299198788609 | 978-2253174561 | 2020-02-28 |    Les Hauts de Hurlevent   |      Le Livre de Poche      |  2012 | 978-2253174561 |
| 533299198788609 | 978-2251013039 | 2020-02-28 |            Énéide           |        Belles Lettres       |  1993 | 978-2251013039 |
| 917547585216771 | 978-2290105504 | 2020-04-07 |  Jack Barron et l'Éternité  |           J'ai Lu           |  2016 | 978-2290105504 |
| 654834075188732 | 978-2864973270 | 2020-02-17 | Astérix et la Transitalique |     Editions Albert René    |  2017 | 978-2864973270 |
| 654834075188732 | 978-2070406340 | 2020-02-17 |   Les Contes de Canterbury  |     Gallimard Education     |  2000 | 978-2070406340 |
| 654834075188732 | 978-2806231697 | 2020-02-17 |        Le Père Goriot       |           Primento          |  2012 | 978-2806231697 |
| 934701281931582 | 978-2260019183 | 2020-01-01 |    La Planète des singes    |           Julliard          |  2011 | 978-2260019183 |
| 934701281931582 | 978-2371240087 | 2020-01-01 |        Anna Karénine        | Bibliothèque russe et slave |  2018 | 978-2371240087 |
|  35184062854281 | 978-2745989857 | 2020-02-18 |          Les Robots         |        Editions Milan       |  2017 | 978-2745989857 |
|  35184062854281 | 978-2072762093 | 2020-02-18 |   Le Vieil Homme et la Mer  |      Editions Gallimard     |  2018 | 978-2072762093 |
|  35184062854281 | 978-2742744824 | 2020-02-18 |    Les Fils de la Médina    |  Arles [France] : Actes sud |  2003 | 978-2742744824 |


```
Result: 17 enregistrements ramenés en 15ms
```

Cette [requête](Fichiers/resultat_11.csv) crée la jointure des deux tables.
Comme on peut le voir, toutes les colonnes des deux tables ont été recopiées dans la sortie.  
Chaque ligne est le résultat de la fusion de deux lignes ayant le même ISBN. Le choix de ces lignes est donné
par la condition de jointure indiquée par le mot clé `ON`. La condition indique au SGBD dans quel cas deux lignes doivent être fusionnées.  
Ici, on joint les lignes pour lesquelles les ISBN sont égaux. On écrit donc l'expression booléenne `emprunt.isbn = livre.isbn`. La notation `nom_de_table.attribut` permet de différencier entre deux attributs portant le même nom. 

La jointure peut être combinée avec les clauses `SELECT` et `WHERE`.  
Par exemple, si on souhaite afficher uniquement [les titres et les dates des livres empruntés qui sont à rendre avant le 1 er février 2020](Fichiers/resultat_12.csv), on peut écrire la requête suivante :

```sql
SELECT livre.titre, emprunt.retour 
FROM emprunt 
JOIN livre ON emprunt.isbn = livre.isbn 
WHERE emprunt.retour < '2020-02-01';
```

|         titre         |   retour   |
|:---------------------:|:----------:|
| La Planète des singes | 2020-01-01 |
| Anna Karénine         | 2020-01-01 |

```
Result: 2 enregistrements ramenés en 9ms
```

Même s'il n'y a pas d'ambiguïté ici, une bonne pratique consiste à préfixer les noms d'attributs par leur table dès que l'on utilise plus d'une table dans la requête. 

On n'est évidemment pas limité à une seule jointure. Si on souhaite afficher les noms et prénoms des utilisateurs ayant emprunté ces livres, il suffit de joindre la table [`usager`](Fichiers/usager.csv), en rajoutant une nouvelle clause `JOIN ON`, cette fois sur le `code_barre` de l'usager.

```sql
SELECT usager.nom, usager.prenom, livre.titre, emprunt.retour
FROM emprunt
JOIN livre ON emprunt.isbn = livre.isbn
JOIN usager ON usager.code_barre = emprunt.code_barre
WHERE emprunt.retour < '2020-02-01';
```

|  nom  |   prenom  |         titre         |   retour   |
|:-----:|:---------:|:---------------------:|:----------:|
| PETIT | SÉBASTIEN | La Planète des singes | 2020-01-01 |
| PETIT | SÉBASTIEN | Anna Karénine         | 2020-01-01 |

```
Result: 2 enregistrements ramenés en 11ms
```

La [requête](Fichiers/resultat_13.csv) ci-dessus fonctionne parfaitement mais est un peu fastidieuse à écrire.  
Il est possible de créer dans une requête un alias pour un nom de table au moyen du mot clé `AS`, comme pour le renommage de colonne. La requête peut donc être réécrite de la manière suivante :


```sql
SELECT u.nom, u.prenom, l.titre, e.retour
FROM emprunt AS e
JOIN livre AS l ON e.isbn = l.isbn
JOIN usager AS u ON u.code_barre = e.code_barre
WHERE e.retour < '2020-02-01';
```

|  nom  |   prenom  |         titre         |   retour   |
|:-----:|:---------:|:---------------------:|:----------:|
| PETIT | SÉBASTIEN | La Planète des singes | 2020-01-01 |
| PETIT | SÉBASTIEN | Anna Karénine         | 2020-01-01 |

```
Result: 2 enregistrements ramenés en 4ms
```


La jointure est une opération fondamentale des bases de données relationnelles. En effet, la modélisation relationnelle des données impose parfois un découpage des données. Les relations entre ces dernières sont maintenues par des contraintes, notamment les contraintes de référence.  
La jointure permet de reconstituer ce lien, en construisant "à la volée" de grandes tables contenant toutes les informations liées.


#### Nouvelle syntaxe SQL pour les jointures
Considérons la requête 

```sql
SELECT livre.titre, emprunt.retour
FROM emprunt
JOIN livre ON emprunt.isbn = livre.isbn
WHERE emprunt.retour < '2020-02-01';
```

Puisque l'on peut mettre des conditions arbitraires dans la clause `WHERE`, une manière alternative d'écrire la requête est la suivante:

```sql
SELECT livre.titre, emprunt.retour
FROM emprunt, livre
WHERE emprunt.isbn = livre.isbn AND emprunt.retour < '2020-02-01';
```

Pour n'importe quel SGBD, ces deux requêtes sont équivalentes, tant du point de vue du résultat que des performances.  
La seconde version est "l'ancienne" syntaxe SQL, utilisée avant l'introduction du mot clé `JOIN` dans la version du standard de 1992.  
La bonne pratique consiste à privilégier l'utilisation du mot clé `JOIN`.  
En effet, il rend plus lisible les grandes requêtes en séparant clairement prédicats de jointure et filtres
sur les données. Il permet aussi de communiquer clairement l'intention d'effectuer une jointure.  
Enfin, lors d'une utilisation avancée, il permet de changer la méthode de jointure (`OUTER JOIN`, `LEFT OUTER JOIN`, `RIGHT OUTER JOIN`, etc.) tout en conservant la même syntaxe.

## Modification des données
Les données stockées dans un SGBD ne sont a priori pas figées et peuvent être modifiées au cours du temps. Nous allons montrer deux types de modifications pouvant être faites sur les tables: 
* la suppression d'un ensemble de lignes 
* la mise à jour de certains attributs d'un ensemble de lignes.

### Suppression de lignes
L'ordre `DELETE FROM t WHERE c` permet de supprimer de la table `t` toutes les lignes vérifiant la condition `c`.  
Dans l'exemple de notre médiathèque, supposons que l'utilisateur Sébastien Petit, dont le `code_barre`
est `'934701281931582'`, ait rendu ses livres.  
Il faut supprimer de la table `emprunt` toutes les lignes pour lesquelles le `code_barre` vaut `934701281931582`, ce qui donne l'ordre suivant:

```sql
DELETE FROM emprunt WHERE code_barre = '934701281931582';
```

```
Result: Requête exécutée avec succès. A pris 1 ms , 2 enregistrements affectés
```

Après exécution de cet ordre, la recherche dans la table emprunt ne donne plus de résultats :

```sql
SELECT COUNT(*) AS total
FROM emprunt
WHERE code_barre = '934701281931582';
```

|            total            |
|:---------------------------:|
| 0                       |

```
Result: 1 enregistrements ramenés en 2ms
```


Attention, au même titre qu'une requête `SELECT` sans clause `WHERE` sélectionne toutes les lignes, un ordre `DELETE` sans clause `WHERE` efface toutes les lignes de la table.  
Il ne faut pas confondre `DELETE FROM t` et `DROP TABLE t`. La première opération vide une table de son contenu, mais ne supprime pas la table. Il est donc possible d'y ajouter de nouveau des données au moyen de l'instruction `INSERT`. La seconde opération détruit la table (et ses données). La table ne peut donc plus être référencée.

Les contraintes sont vérifiées à chaque mise à jour.  
Essayons de supprimer le livre `Hacker's delight` de la table `livre`, sachant que l'ISBN de ce dernier est `'978-0201914658'`.

```sql
DELETE FROM livre WHERE isbn = '978-0201914658';
```

```
Result: FOREIGN KEY constraint failed
```

Ici, le SGBD nous indique que supprimer ce livre (et donc supprimer sa clé primaire de la table livre) violerait la contrainte de clé étrangère dans la table `auteur_de`.  
Comme pour la destruction d'une table, il faut donc supprimer en premier les lignes dont les attributs sont déclarés comme clés étrangères avant de supprimer celles contenant les clés primaires correspondantes.

Du point de vue de leur exécution, les ordres de modification de table sont soit entièrement exécutés, soit entièrement annulés. Considérons la requête suivante:

```sql
DELETE FROM usager WHERE cp = '75001' OR cp = '75002';
```

```
Result: Requête exécutée avec succès. A pris 0 ms , 0 enregistrements affectés
```

qui efface de la table usager toutes les personnes dont le code postal est 75001 ou 75002. Si aucune de ces personnes n'apparaît dans la table `emprunt`, alors les suppressions peuvent être effectuées sans erreur. Supposons maintenant que certaines de ces personnes ont emprunté un livre. Même si le SGDB rencontre en premier des personnes sans emprunt et les supprime, il lèvera une erreur dès qu'il rencontrera un usager référencé dans la table `emprunt`. Dans ce cas, *toutes les modifications déjà faites seront annulées* et
la table se trouvera dans l'état qu'elle avait avant la tentative d'exécution.  
Les exécutions sont donc de type « tout ou rien ».



### Mise à jour
Le second type de modification est la mise à jour. Elle consiste à remplacer certains attributs d'un ensemble de lignes par de nouvelles valeurs.
La syntaxe est la suivante :

```
UPDATE t SET a_1 = e_1, ..., SET a_n = e_n WHERE c
```
Cette dernière signifie *sélectionne dans la table `t` toutes les lignes vérifiant la condition `c` et, pour chacune de ces lignes, remplace la valeur courante de l'attribut `a_i` par la valeur de l'expression `e_i`*.   
Par exemple, si l'utilisateur Sébastien Petit souhaite mettre à jour son adresse email, on écrit ceci:

```sql
UPDATE usager SET email = 'spetit@hmail.com'
WHERE code_barre = '934701281931582';
```

```
Result: Requête exécutée avec succès. A pris 0 ms , 1 enregistrements affectés
```

Les expressions de mise à jour peuvent mentionner des noms d'attributs.  
Ces derniers sont alors remplacés par la valeur courante (avant mise à jour) de ces attributs. Supposons par exemple que la médiathèque soit fermée au mois d'avril. On souhaite que tous les emprunts dont la date de rendu était en avril soient prolongés de 30 jours.


```sql
UPDATE emprunt SET retour = retour + 30
WHERE retour >= '2020-04-01';
```

```
Result: Requête exécutée avec succès. A pris 0 ms , 4 enregistrements affectés
```

Dans la mise à jour précédente, la clause `SET retour = retour + 30` est similaire à la modification d'une variable dans un langage de programmation comme Python, c'est-à-dire prendre la valeur courante de retour, y
ajouter 30 et écrire la nouvelle valeur dans retour.


### Copie de table
Toute modification (création de table, suppression de table, mise à jour, suppression de ligne, insertion de ligne) qui ne viole pas de contrainte est définitive. En cas de suppression ou de mise à jour, les anciennes données sont perdues.  
Il faut donc être particulièrement vigilant lors de la conception d'un programme effectuant des mises à jour dans une base de données.  
Les bonnes pratiques recommandent l'utilisation de plusieurs "copies" de la base de données. Une copie de test, utilisée pour le développement et une copie de "production" utilisée pour faire fonctionner le logiciel, une fois que ce dernier a été testé rigoureusement. La base de production doit par ailleurs être sauvegardée fréquemment pour éviter les risques liés à de mauvaises manipulations ou à des défaillances logicielles ou matérielles.

Créer une copie conforme d'une table peut se faire en utilisant deux variations sur des opérations que nous avons rencontré. 

```sql
CREATE TABLE usager3 AS SELECT * FROM usager;
```


## Requêtes imbriquées
L'opération `SELECT ... INTO` permet de sauver le résultat d'une requête sous un certain nom de table. Il est donc possible d'effectuer sur ce résultat une nouvelle requête.  
Cependant, cette opération va occuper de l'espace de stockage. Il ne faudra donc pas oublier de supprimer la table ainsi créée. Il est possible de créer une table de manière temporaire et d'exécuter une requête sur cette table en imbriquant la première requête dans la clause `FROM` de la seconde ou dans une clause `JOIN ... ON` :

```sql
SELECT * FROM (SELECT * FROM livre WHERE annee >= 1990) AS tmp
WHERE tmp.annee <= 2000;
```

```
Result: 18 enregistrements ramenés en 9ms
```

La [requête](Fichiers/resultat_14.csv) ci-dessus calcule d'abord une table intermédiaire nommée `tmp` qui
liste les livres publiés après 1990.  
Suite à quoi, la table `tmp` est refiltrée pour ne garder que les livres pour lesquels l'année est inférieure à 2000.  
Attention, il ne s'agit ici que d'une explication de "haut niveau". En pratique, n'importe quel SGBD moderne évaluera cette deux requêtes imbriquées comme la requête équivalente :

```sql
SELECT * FROM livre WHERE annee >= 1990 AND annee <= 2000;
```

```
Result: 18 enregistrements ramenés en 10ms
```

Une autre manière d'imbriquer les requêtes consiste à utiliser une sous-requête dans la clause `WHERE`. En effet, le langage SQL identifie les valeurs scalaires et les tables à une seule "case" telles que celles renvoyées par les fonctions d'agrégation.  
Par exemple, si on souhaite afficher [les titres des livres dont l'année est la plus ancienne dans la base](Fichiers/resultat_15.csv), on pourra écrire:

```sql
SELECT titre FROM livre WHERE annee = (SELECT MIN(annee) FROM livre);
```

|            titre            |
|:---------------------------:|
| Berlin Alexanderplatz                      |


```
Result: 1 enregistrements ramenés en 2ms
```


Ici, la sous-requête calcule l'année minimum de la table `livre` (1933 dans notre base), puis affiche tous les titres de livres dont l'année vaut 1933.  
Attention, la sous-requête ne doit pas nécessairement comporter une fonction d'agrégation. Il suffit qu'elle renvoie une table contenant une seule valeur.  
Ainsi, si nous voulons afficher [les titres des livres publiés la même année que Moby Dick](Fichiers/resultat_16.csv) (sans connaître cette année), nous pouvons écrire:

```sql
SELECT titre FROM livre WHERE annee = (SELECT annee FROM livre WHERE titre = 'Moby Dick');
```

|             titre             |
|:-----------------------------:|
| Moby Dick                     |
| L'Île des morts               |
| Le Devin                      |
| Le Berceau du chat            |
| Les Enfants de minuit         |
| À la recherche du temps perdu |


```
Result: 6 enregistrements ramenés en 13ms
```


Un opérateur utilisant la puissance des requêtes imbriquées est l'opérateur `IN`. L'expression `e IN (q)` renvoie vrai si et seulement si la valeur résultant de l'évaluation de `e` est l'une des lignes renvoyées par la requête `q`. Ainsi, pour exprimer la requête [afficher les titres des livres qui ont été publiés la
même année qu'un livre dont le titre contient Astérix](Fichiers/resultat_17.csv), on pourra écrire

```sql
SELECT titre FROM livres WHERE annee IN
(SELECT annee FROM livres WHERE titre LIKE '%Astérix%');
```

```
Result: 38 enregistrements ramenés en 6ms
```

#### Requêtes de groupe
Leur compréhension peut aider lors de la création d'exercices, en particulier pour déterminer sî la requête dêmandée.
Ces requêtes s'expriment au moyen de l'opérateur `GROUP BY` (éventuellement accompagné de l'opérateur `HAVING`).  
Intuitivement, ces requêtes permettent de répondre à la question `donner le f de x pour chaque g distinct de la table t` où `f` est une fonction d'agrégation, `x` un attribut de la table et `g` un autre attribut appelé clé de groupe.  

Cette requête s'écrira alors

```sql
SELECT g, f(x) FROM t GROUP BY g;
```

Par exemple, si on souhaite connaître [le nombre (f) de livres (t) publiés pour chaque année (g) de la base](Fichiers/resultat_18.csv), on écrira :

```sql
SELECT annee, COUNT (*) FROM livre GROUP BY annee;
```

| annee | COUNT (*) |
|:-----:|:---------:|
|  1933 |     1     |
| 1959  | 1         |
| 1967  | 1         |
| 1971  | 2         |
| 1972  | 2         |
| 1974  | 2         |
| 1978  | 1         |
| 1979  | 2         |
| 1980  | 1         |
| 1983  | 2         |
| 1988  | 1         |
| ...   |           |

```
Result: 38 enregistrements ramenés en 12ms
```


## Exercices
### Exercice 1 (requêtes simples, sans jointure ni imbrication)  
Soit la base de données de la médiatèque : [mediatheque.sql](Fichiers/mediatheque.sql) et [mediatheque.db](Fichiers/mediatheque.db).   
Donner le code SQL de chacune des requêtes ci-dessous.  
Les mots en `police fixe` donnent une indication sur les attributs et les tables à utiliser dans la requête.
1. Tous les `titre`s de `livre`.
2. Tous les `nom`s d'`usager`.
3. Tous les `nom`s d'`usager` en retirant les doublons.
4. Les `titre`s des livres publiés avant 1980.
5. Les `titre`s des livres dont le titre contient la lettre `'A'`.
6. Les `isbn` des livres à rendre pour le 01/01/2020.
7. Les `nom`s d'`auteur`s triés par ordre alphabétique.
8. Les `nom`s d'`usager`s vivant dans le 12 e ou 13 e arrondissement de Paris (codes postaux 75012 et 75013).
9. Les `nom`s et `adresse`s des `usager`s n'habitant pas dans une rue. (la chaîne `'Rue'` ne doit pas apparaître dans l'adresse).
10. Les `annee`s et `titre`s des `livre`s publiés lors d'une année bissextile.
On rappelle que ce sont les années divisibles par 4, mais pas celles divisibles par 100 sauf si elles sont divisibles par 400.

### Exercice 2 (requêtes avancées avec jointure ou imbrication)  
Soit la base de données de la médiatèque : [mediatheque.sql](Fichiers/mediatheque.sql) et [mediatheque.db](Fichiers/mediatheque.db).  
Donner le code SQL de chacune des requêtes ci-dessous.  
Les mots en `police fixe` donnent une indication sur les attributs et les tables à utiliser dans la requête.
1. Le `titre` des `livre`s empruntés.
2. Le `titre` des `livre`s empruntés à rendre avant le 31/03/2020.
3. Le `nom` et `prenom` de l'auteur du livre `'1984'`.
4. Le `nom` et le `prenom` des `usager`s ayant `emprunt`é des livres, sans doublons (i. e. si un usager a emprunté plusieurs livres, il ne doit apparaître qu'une fois dans le résultat).
5. Même requête que précédemment, avec les noms triés par ordre alphabétique.
6. Les `titre` des `livre`s publiés strictement avant `'Dune'`.
7. Les `nom`s et `prenom`s des `auteur`s des `livre`s trouvés à la question précédente.
8. Comme la question précédente, en retirant les doublons.
9. Le nombre de résultats trouvés à la question précédente.

### Exercice 3
Soit la base de données de la médiatèque : [mediatheque.sql](Fichiers/mediatheque.sql) et [mediatheque.db](Fichiers/mediatheque.db).   
Formuler simplement en francais les requêtes SQL suivantes.

1. 
```sql
SELECT * FROM livre WHERE titre LIKE '%Robot%';
```
2. 
```sql
SELECT nom, prenom FROM usager WHERE ville = 'Guingamp';
```
3. 
```sql
SELECT u.nom, u.prenom
FROM usager AS u
JOIN emprunt AS e ON u.code_barre = e.code_barre
WHERE retour < '2020-04-02';
```
4. 
```sql
SELECT l.titre
FROM livre AS l
WHERE l.isbn IN (SELECT isbn FROM livre WHERE annee > 1990);
```

Réécrire la requête 4 de façon à utiliser une seule clause `SELECT`.

### Exercice 4
Soit la base de données de la médiatèque : [mediatheque.sql](Fichiers/mediatheque.sql) et [mediatheque.db](Fichiers/mediatheque.db).  
Calculer tous les auteurs ayant collaboré sur un ouvrage et les renvoyer sous la forme $(n_1, p_1, n_2,p_2, t)$ où les $n_i$ sont les noms des auteurs, $p_i$ leur prénoms et $t$ le titre du livre sur lequel ils ont collaboré. Si trois auteurs ont collaboré sur le même livre, on souhaite avoir trois lignes de résultats $(auteur1/auteur2, auteur2/auteur3 et auteur1/auteur3)$ et non pas les trois sur la même ligne.  
Pour ne pas afficher deux fois le même couple, on demande en plus que $n_1 < n_2$.

### Exercice 5
On considère les trois tables décrites ci-dessous. 


```sql
CREATE TABLE x (a INT PRIMARY KEY, b INT, CHECK (b >= 0));
CREATE TABLE y (c INT PRIMARY KEY, d INT, CHECK (d <= 30));
CREATE TABLE z (a INT REFERENCES x(a), c INT REFERENCES y(c), e INT, UNIQUE (a,c));
```

* Table `x` :

| a | b |
|:-:|:-:|
| 1 | 1 |
| 2 | 2 |
| 3 | 2 |
| 4 | 2 |
| 5 | 1 |
| 6 | 9 |
| 7 | 1 |

* Table `y` :

|  c |  d |
|:--:|:--:|
|  9 |  9 |
| 10 | 10 |
| 11 |  9 |
| 12 | 20 |
| 13 | 30 |
| 14 |  9 |
| 15 | 1  |
| 16 | 10 |
| 17 | 10 |

* Table `z` :

| a |  c |  e |
|:-:|:--:|:--:|
| 1 | 11 | 30 |
| 2 | 14 | 9  |
| 5 | 15 | 1  |
| 7 | 17 | 3  |
| 1 | 10 | 50 |
| 2 | 9  | 8  |
| 2 | 15 | 15 |
| 3 | 17 | 19 |
| 4 | 16 | 12 |
| 5 | 10 | 20 |
| 2 | 11 | 30 |
| 7 | 14 | 9  |
| 7 | 9  | 12 |


Pour chacune des requêtes SQL ci-dessous, calculer son résultat (à la main).

1. 
```sql
SELECT * FROM x WHERE b > 3;
```
2. 
```sql
SELECT DISTINCT e FROM z
WHERE e > 10 AND e < 50;
```
3. 
```sql
SELECT * FROM y WHERE c % 2 = 0 ORDER BY d ASC;
```
4. 
```sql
SELECT x.a,x.b FROM x
JOIN z ON z.a = x.a
WHERE z.e < 9;
```
5. 
```sql
SELECT DISTINCT x.b,y.d FROM x
JOIN z ON z.a = x.a
```


### Exercice 6
On considère les trois tables décrites ci-dessous. 

```sql
CREATE TABLE x (a INT PRIMARY KEY, b INT, CHECK (b >= 0));
CREATE TABLE y (c INT PRIMARY KEY, d INT, CHECK (d <= 30));
CREATE TABLE z (a INT REFERENCES x(a), c INT REFERENCES y(c), e INT, UNIQUE (a,c));
```

* Table `x` :

| a | b |
|:-:|:-:|
| 1 | 1 |
| 2 | 2 |
| 3 | 2 |
| 4 | 2 |
| 5 | 1 |
| 6 | 9 |
| 7 | 1 |

* Table `y` :

|  c |  d |
|:--:|:--:|
|  9 |  9 |
| 10 | 10 |
| 11 |  9 |
| 12 | 20 |
| 13 | 30 |
| 14 |  9 |
| 15 | 1  |
| 16 | 10 |
| 17 | 10 |

* Table `z` :

| a |  c |  e |
|:-:|:--:|:--:|
| 1 | 11 | 30 |
| 2 | 14 | 9  |
| 5 | 15 | 1  |
| 7 | 17 | 3  |
| 1 | 10 | 50 |
| 2 | 9  | 8  |
| 2 | 15 | 15 |
| 3 | 17 | 19 |
| 4 | 16 | 12 |
| 5 | 10 | 20 |
| 2 | 11 | 30 |
| 7 | 14 | 9  |
| 7 | 9  | 12 |

Pour chacune des modifications ci-dessous, indiquer si elle réussit ou si elle échoue.  
* Si elle réussit, indiquer comment la table est modifiée.  
* Si elle échoue, expliquer pourquoi. 

Les questions sont indépendantes, c'est-à-dire que chacune repart des tables de l'énoncé entre chaque question.

1. 
```sql
UPDATE x SET b = b + a;
```
2. 
```sql
UPDATE x SET b = b - 2;
```
3. 
```sql
INSERT INTO z VALUES (1, 17, 1);
```
4. 
```sql
INSERT INTO z VALUES (1, 18, 1);
```
5. 
```sql
INSERT INTO z VALUES (1, 10, 1);
```
6. 
```sql
DELETE FROM y WHERE c >= 12 AND c <= 13;
```
7. 
```sql
DELETE FROM y WHERE c >= 12 AND c <= 14;
```
8. 
```sql
INSERT INTO y VALUES (40, 20);
```
9. 
```sql
INSERT INTO y VALUES (20, 40);
```
10. 
```sql
DELETE FROM z WHERE a % 2 = 0 OR c % 2 = 0 OR e % 2 = 0;
```

## Sources :
* Balabonski Thibaut, et al. 2020. *Spécialité Numérique et sciences informatiques : 24 leçons avec exercices corrigés - Terminale - Nouveaux programmes*. Paris. Ellipse
* [SQLite](https://www.sqlite.org/index.html)
* [DB Browser for SQLite](https://sqlitebrowser.org/)
* Base de données de la médiatèque : [mediatheque.sql](Fichiers/mediatheque.sql) et [mediatheque.db](Fichiers/mediatheque.db)
* [SQL Murder Mystery](https://mystery.knightlab.com/) : jeu en ligne. Utiliser les requètes SQL pour resoudre une énigme.